In [1]:
import calendar
import os
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine
#from pandas.tseries.offsets import BDay
from pandas.tseries.offsets import * 

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

pd.options.display.max_rows = 11
pd.options.display.float_format = '{:.2f}'.format

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
osd_path = "\\Users\\User\\OneDrive\\Documents\\obsidian-git-sync\\Data\\"

today = date.today()
yesterday = today - timedelta(days=1)
today, yesterday

(datetime.date(2023, 6, 12), datetime.date(2023, 6, 11))

In [2]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
print(f'today: {today}')
to_mmdd = today.strftime('%m-%d')
print(f'Today in mm-dd format: {to_mmdd}')
print(f'yesterday: {yesterday}')

today: 2023-06-12
Today in mm-dd format: 06-12
yesterday: 2023-06-09 00:00:00


In [3]:
#yesterday = date(2023, 3, 3)
yesterday

Timestamp('2023-06-09 00:00:00')

In [4]:
#yesterday = yesterday.date()
one_month_ago = yesterday + DateOffset(days = -28)
print('One month ago: ', one_month_ago, ' ', one_month_ago.day_name())

One month ago:  2023-05-12 00:00:00   Friday


In [5]:
format_dict = {
    'ttl_qty':'{:,}','qty':'{:,}',
    'from_price':'{:.2f}','to_price':'{:.2f}','diff':'{:.2f}','fm_price':'{:.2f}',
    'price':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}',
    'maxp':'{:.2f}','minp':'{:.2f}','pct':'{:,.2f}%',  
}

cols = 'name fm_md to_md days fm_price to_price pct spd avg max_price min_price ttl_qty trend'.split()

### Get past one quarter data

In [6]:
sql = """
SELECT name
FROM buy 
ORDER BY name
"""
buy = pd.read_sql(sql, const)

names = buy['name'].values.tolist()
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

"'AH', 'ASK', 'ASP', 'BANPU', 'BCH', 'CPNREIT', 'DIF', 'GVREIT', 'IVL', 'JASIF', 'JMART', 'JMT', 'KCE', 'MCS', 'NER', 'ORI', 'PTTGC', 'RCL', 'SCC', 'SENA', 'SINGER', 'STA', 'SYNEX', 'TFFIF', 'TMT', 'WHAIR', 'WHART'"

In [7]:
in_p = in_p + ", 'AWC', 'BEM', 'EA', 'KTC'"
in_p

"'AH', 'ASK', 'ASP', 'BANPU', 'BCH', 'CPNREIT', 'DIF', 'GVREIT', 'IVL', 'JASIF', 'JMART', 'JMT', 'KCE', 'MCS', 'NER', 'ORI', 'PTTGC', 'RCL', 'SCC', 'SENA', 'SINGER', 'STA', 'SYNEX', 'TFFIF', 'TMT', 'WHAIR', 'WHART', 'AWC', 'BEM', 'EA', 'KTC'"

### Bypass this statement if aboce statement is executed

In [8]:
sql = """
SELECT name
FROM orders
ORDER BY trade, name
"""
#buy = pd.read_sql(sql, conlite)

#names = buy['name'].values.tolist()
#in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

"'ASK', 'ASP', 'BANPU', 'BCH', 'CPNREIT', 'DIF', 'GVREIT', 'IVL', 'JASIF', 'JMART', 'JMT', 'KCE', 'MCS', 'NER', 'ORI', 'PTTGC', 'RCL', 'SCC', 'SENA', 'SINGER', 'STA', 'SYNEX', 'TFFIF', 'TMT', 'WHAIR', 'WHART', 'AWC', 'GULF', 'TTB'"

### End of Selection between Buy or Orders

In [8]:
sql = """
SELECT name, date, price, qty, maxp, minp
FROM price 
WHERE date >= '%s' AND name IN (%s) 
ORDER BY name, date"""
sql = sql % (one_month_ago, in_p)
print(sql)


SELECT name, date, price, qty, maxp, minp
FROM price 
WHERE date >= '2023-05-12 00:00:00' AND name IN ('AH', 'ASK', 'ASP', 'BANPU', 'BCH', 'CPNREIT', 'DIF', 'GVREIT', 'IVL', 'JASIF', 'JMART', 'JMT', 'KCE', 'MCS', 'NER', 'ORI', 'PTTGC', 'RCL', 'SCC', 'SENA', 'SINGER', 'STA', 'SYNEX', 'TFFIF', 'TMT', 'WHAIR', 'WHART', 'AWC', 'BEM', 'EA', 'KTC') 
ORDER BY name, date


In [9]:
price = pd.read_sql(sql, const)
price.sample(5)

,name,date,price,qty,maxp,minp
549,SYNEX,2023-05-17,12.70,208995,13.10,12.70
533,STA,2023-05-24,19.00,1715946,19.30,19.00
83,AWC,2023-06-12,5.10,24839616,5.15,5.00
295,JMT,2023-05-15,40.25,16249580,42.75,39.75
94,BANPU,2023-05-26,8.10,106238031,8.15,7.90


In [10]:
data_path = "../data/"
file_name = "Quarterly-Price-by-Name.csv"
output_file = data_path + file_name
osd_file = osd_path + file_name

price.to_csv(output_file, header=None, index=False)
price.to_csv(osd_file, header=True, index=False)

### Call ruby ruby\crt-summary.rb

In [11]:
%pwd

'C:\\Users\\User\\OneDrive\\A7\\Daily'

In [12]:
os.chdir("C:\\users\\user\\onedrive\\a7")
%pwd

'C:\\users\\user\\onedrive\\a7'

In [13]:
!ruby ruby\\crt-summary.rb

ruby/../Data/Quarterly-Price-by-Name.csv
name|fm_date|to_date|days|fm_price|to_price|diff|pct|spd|avg|max_price|min_price|trend|nbr|ttl_qty
AH|2023-05-12|2023-05-12|1|33.75|33.75|0.0|0.0|0|0|35.0|33.5||2|6,034,265
AH|2023-05-15|2023-05-15|1|33.25|33.25|0.0|0.0|0|0|34.0|32.5|Downward|3|2,207,306
AH|2023-05-16|2023-05-16|1|35.5|35.5|0.0|0.0|0|0|35.5|33.0|Upward|4|5,526,136
AH|2023-05-17|2023-05-19|3|35.25|34.25|-1.0|-2.84|-4|-2|36.25|34.25|Downward|7|6,825,646
AH|2023-05-22|2023-05-23|2|34.5|34.75|0.25|0.72|1|0|35.0|32.75|Upward|9|2,820,716
AH|2023-05-24|2023-05-25|2|34.0|34.0|0.0|0.0|0|0|35.0|33.75|Downward|11|1,840,670
AH|2023-05-26|2023-05-31|4|34.75|37.75|3.0|8.63|12|3|38.25|34.0|Upward|15|14,202,687
AH|2023-06-01|2023-06-01|1|37.0|37.0|0.0|0.0|0|0|37.75|36.5|Downward|16|2,882,866
AH|2023-06-02|2023-06-07|4|37.25|39.25|2.0|5.37|8|2|39.25|36.25|Upward|19|8,078,969
AH|2023-06-08|2023-06-09|2|36.5|34.5|-2.0|-5.48|-8|-4|37.75|27.75|Downward|21|103,830,264
AH|2023-06-12|2023-06-12|1|35.25

STA|2023-06-06|2023-06-07|2|18.7|18.7|0.0|0.0|0|0|19.0|18.5|Downward|544|6,930,076
STA|2023-06-08|2023-06-08|1|19.0|19.0|0.0|0.0|0|0|19.1|18.6|Upward|545|5,394,096
STA|2023-06-09|2023-06-12|2|18.8|18.5|-0.3|-1.6|-3|-2|19.0|18.4|Downward|547|10,408,367
SYNEX|2023-05-12|2023-05-15|2|12.7|12.7|0.0|0.0|0|0|13.1|12.5|Downward|549|1,610,020
SYNEX|2023-05-16|2023-05-16|1|13.0|13.0|0.0|0.0|0|0|13.1|12.6|Upward|550|392,827
SYNEX|2023-05-17|2023-05-17|1|12.7|12.7|0.0|0.0|0|0|13.1|12.7|Downward|551|208,995
SYNEX|2023-05-18|2023-05-18|1|12.9|12.9|0.0|0.0|0|0|13.0|12.8|Upward|552|262,424
SYNEX|2023-05-19|2023-05-19|1|12.7|12.7|0.0|0.0|0|0|12.9|12.7|Downward|553|166,247
SYNEX|2023-05-22|2023-05-25|4|12.9|13.2|0.3|2.33|3|0|13.3|12.3|Upward|557|1,038,244
SYNEX|2023-05-26|2023-05-26|1|13.0|13.0|0.0|0.0|0|0|13.3|13.0|Downward|558|264,960
SYNEX|2023-05-29|2023-05-31|3|13.1|13.3|0.2|1.53|2|0|13.4|12.9|Upward|561|747,379
SYNEX|2023-06-01|2023-06-02|2|13.0|12.9|-0.1|-0.77|-1|-1|13.2|12.9|Downward|563|375,58

In [14]:
os.chdir("C:\\users\\user\\onedrive\\a7\\daily")
%pwd

'C:\\users\\user\\onedrive\\a7\\daily'

In [15]:
file_name   = 'summary.csv'
input_file = data_path + file_name
df = pd.read_csv(input_file, sep=',', index_col=None)
df.sample(5).style.format(format_dict)

,name,from_date,to_date,days,from_price,to_price,diff,pct,spd,avg,max_price,min_price,ttl_qty,trend,nbr
157,KTC,2023-06-06,2023-06-06,1,50.75,50.75,0.00,0.00%,0,0,51.75,50.25,"6,263,735",Downward,354
51,BCH,2023-06-06,2023-06-08,3,18.40,18.40,0.00,0.00%,0,0,18.70,18.10,"35,910,721",Upward,125
223,SINGER,2023-06-01,2023-06-07,5,11.30,11.00,-0.30,-2.65%,-3,-1,11.60,10.80,"21,823,850",Downward,523
97,GVREIT,2023-06-12,2023-06-12,1,9.00,9.00,0.00,0.00%,0,0,9.05,9.00,"545,500",Downward,232
158,KTC,2023-06-07,2023-06-09,3,51.00,53.00,2.00,3.92%,8,2,53.00,50.00,"15,022,617",Upward,357


In [16]:
df.rename(columns={'from_date':'fm_date','from_price':'fm_price'},inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df['fm_md'] = df['fm_date'].dt.strftime('%m-%d')
df['to_md'] = df['to_date'].dt.strftime('%m-%d')

### Group analysis

In [17]:
mask_p = df['to_md'] == to_mmdd
df[mask_p][cols].sort_values('pct',ascending=False).head().style.format(format_dict)

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
138,JMT,06-07,06-12,4,39.50,43.25,9.49%,15,3,44.25,38.50,"69,435,297",Upward
42,BANPU,06-02,06-12,7,8.30,8.75,5.42%,9,1,8.80,8.20,"549,071,452",Upward
172,NER,06-07,06-12,4,4.48,4.68,4.46%,10,2,4.70,4.44,"26,686,425",Upward
127,JMART,06-08,06-12,3,19.30,19.90,3.11%,6,2,20.30,18.90,"54,411,471",Upward
197,RCL,06-02,06-12,7,23.70,24.40,2.95%,7,1,24.80,23.30,"22,125,042",Upward


In [18]:
name_up = 'JMT'
mask_n = df.name == name_up
df[mask_n][cols].style.format(format_dict)

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
128,JMT,05-12,05-12,1,42.75,42.75,0.00%,0,0,42.75,38.50,"21,067,914",Upward
129,JMT,05-15,05-15,1,40.25,40.25,0.00%,0,0,42.75,39.75,"16,249,580",Downward
130,JMT,05-16,05-19,4,41.25,43.00,4.24%,7,1,43.25,40.00,"37,647,453",Upward
131,JMT,05-22,05-24,3,42.75,41.50,-2.92%,-5,-2,43.25,41.25,"19,971,612",Downward
132,JMT,05-25,05-25,1,42.00,42.00,0.00%,0,0,42.25,40.50,"8,596,495",Upward
133,JMT,05-26,05-26,1,40.50,40.50,0.00%,0,0,42.25,40.50,"6,536,809",Downward
134,JMT,05-29,05-29,1,40.75,40.75,0.00%,0,0,41.75,40.25,"5,354,030",Upward
135,JMT,05-30,05-30,1,39.75,39.75,0.00%,0,0,40.75,39.50,"11,998,586",Downward
136,JMT,05-31,05-31,1,43.00,43.00,0.00%,0,0,44.00,39.25,"69,063,646",Upward
137,JMT,06-01,06-06,4,42.50,39.25,-7.65%,-13,-4,43.25,39.25,"54,350,987",Downward


In [19]:
mask_m = df['to_md'] == to_mmdd
df[mask_m][cols].sort_values('pct',ascending=True).head().style.format(format_dict)

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
234,STA,06-09,06-12,2,18.80,18.50,-1.60%,-3,-2,19.00,18.40,"10,408,367",Downward
88,EA,06-09,06-12,2,64.00,63.75,-0.39%,-1,-1,65.00,63.50,"14,588,407",Downward
207,SCC,06-09,06-12,2,333.00,332.00,-0.30%,-1,-1,337.00,331.00,"2,647,359",Downward
10,AH,06-12,06-12,1,35.25,35.25,0.00%,0,0,36.25,34.75,"7,101,473",Upward
259,TMT,06-12,06-12,1,7.45,7.45,0.00%,0,0,7.50,7.45,"16,180",Downward


In [20]:
name_down = 'STA'
mask_n = df.name == name_down
df[mask_n][cols].style.format(format_dict)

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
226,STA,05-12,05-15,2,19.30,18.50,-4.15%,-8,-4,19.60,18.40,"12,157,529",Downward
227,STA,05-16,05-23,6,18.70,19.20,2.67%,5,0,19.30,18.10,"23,096,040",Upward
228,STA,05-24,05-29,4,19.00,18.90,-0.53%,-1,-1,19.30,18.70,"9,800,208",Downward
229,STA,05-30,05-30,1,19.10,19.10,0.00%,0,0,19.20,18.70,"2,752,929",Upward
230,STA,05-31,06-01,2,19.00,18.70,-1.58%,-3,-2,19.00,18.60,"6,592,379",Downward
231,STA,06-02,06-02,1,18.90,18.90,0.00%,0,0,19.00,18.80,"2,003,229",Upward
232,STA,06-06,06-07,2,18.70,18.70,0.00%,0,0,19.00,18.50,"6,930,076",Downward
233,STA,06-08,06-08,1,19.00,19.00,0.00%,0,0,19.10,18.60,"5,394,096",Upward
234,STA,06-09,06-12,2,18.80,18.50,-1.60%,-3,-2,19.00,18.40,"10,408,367",Downward


In [21]:
df_groupby_trend = df.groupby('trend')

In [22]:
# To retrieve one of the created groups
df_up = df_groupby_trend.get_group('Upward')
df_up[cols].sample(5).style.format(format_dict)

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
103,IVL,05-30,05-30,1,32.50,32.50,0.00%,0,0,32.50,31.75,"9,710,381",Upward
202,SCC,05-23,05-23,1,337.00,337.00,0.00%,0,0,339.00,333.00,"1,303,777",Upward
127,JMART,06-08,06-12,3,19.30,19.90,3.11%,6,2,20.30,18.90,"54,411,471",Upward
22,ASP,05-23,05-23,1,2.86,2.86,0.00%,0,0,2.88,2.84,"916,018",Upward
58,BEM,05-29,05-30,2,8.15,8.15,0.00%,0,0,8.20,8.00,"53,854,830",Upward


In [23]:
# To retrieve one of the created groups
df_dw = df_groupby_trend.get_group('Downward')
df_dw[cols].sample(5).style.format(format_dict)

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
194,RCL,05-25,05-26,2,24.40,24.20,-0.82%,-2,-1,24.70,24.20,"3,475,739",Downward
79,DIF,06-09,06-12,2,12.30,12.30,0.00%,0,0,12.40,12.20,"17,355,872",Downward
28,AWC,05-12,05-22,7,5.45,5.15,-5.50%,-6,-1,5.55,5.00,"226,827,220",Downward
48,BCH,05-24,05-26,3,17.90,17.70,-1.12%,-2,-1,18.20,17.40,"54,046,737",Downward
162,MCS,05-25,05-31,5,7.35,7.10,-3.40%,-5,-1,7.40,7.00,"1,819,764",Downward


In [24]:
# there is a function called .agg() and it allows us to specify multiple aggregation functions at once
df.groupby('trend').days.agg(['max', 'count', 'median', 'mean'])

,max,count,median,mean
trend,,,,
Downward,10,142,2.00,2.65
Upward,7,134,2.00,2.16


In [25]:
upward_max = df.groupby('trend').days.max().loc['Upward']
downward_max = df.groupby('trend').days.max().loc['Downward']

In [26]:
df.query('days == @upward_max and trend == "Upward"')[cols].style.format(format_dict)

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
42,BANPU,06-02,06-12,7,8.30,8.75,5.42%,9,1,8.80,8.20,"549,071,452",Upward
197,RCL,06-02,06-12,7,23.70,24.40,2.95%,7,1,24.80,23.30,"22,125,042",Upward


In [27]:
df.query('days == @downward_max and trend == "Downward"')[cols]

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
257,TMT,05-26,06-08,10,7.60,7.45,-1.97,-3,-1,7.65,7.40,1082757,Downward


### Individual stock analysis

In [28]:
df.query('name == @name_up')[cols].style.format(format_dict)

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
128,JMT,05-12,05-12,1,42.75,42.75,0.00%,0,0,42.75,38.50,"21,067,914",Upward
129,JMT,05-15,05-15,1,40.25,40.25,0.00%,0,0,42.75,39.75,"16,249,580",Downward
130,JMT,05-16,05-19,4,41.25,43.00,4.24%,7,1,43.25,40.00,"37,647,453",Upward
131,JMT,05-22,05-24,3,42.75,41.50,-2.92%,-5,-2,43.25,41.25,"19,971,612",Downward
132,JMT,05-25,05-25,1,42.00,42.00,0.00%,0,0,42.25,40.50,"8,596,495",Upward
133,JMT,05-26,05-26,1,40.50,40.50,0.00%,0,0,42.25,40.50,"6,536,809",Downward
134,JMT,05-29,05-29,1,40.75,40.75,0.00%,0,0,41.75,40.25,"5,354,030",Upward
135,JMT,05-30,05-30,1,39.75,39.75,0.00%,0,0,40.75,39.50,"11,998,586",Downward
136,JMT,05-31,05-31,1,43.00,43.00,0.00%,0,0,44.00,39.25,"69,063,646",Upward
137,JMT,06-01,06-06,4,42.50,39.25,-7.65%,-13,-4,43.25,39.25,"54,350,987",Downward


In [29]:
df_tmp = price.query('name == @name_up')
df_tmp.describe().round(2)

,price,qty,maxp,minp
count,21.00,21.00,21.00,21.00
mean,41.55,15251067.10,42.45,40.50
std,1.34,14194151.35,1.21,1.30
min,39.25,4947823.00,39.75,38.50
25%,40.50,7454241.00,41.75,39.50
50%,42.00,11998586.00,42.75,40.50
75%,42.75,16068994.00,43.25,41.50
max,43.25,69063646.00,44.25,42.75


In [30]:
df.query('name == @name_down')[cols].style.format(format_dict)

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
226,STA,05-12,05-15,2,19.30,18.50,-4.15%,-8,-4,19.60,18.40,"12,157,529",Downward
227,STA,05-16,05-23,6,18.70,19.20,2.67%,5,0,19.30,18.10,"23,096,040",Upward
228,STA,05-24,05-29,4,19.00,18.90,-0.53%,-1,-1,19.30,18.70,"9,800,208",Downward
229,STA,05-30,05-30,1,19.10,19.10,0.00%,0,0,19.20,18.70,"2,752,929",Upward
230,STA,05-31,06-01,2,19.00,18.70,-1.58%,-3,-2,19.00,18.60,"6,592,379",Downward
231,STA,06-02,06-02,1,18.90,18.90,0.00%,0,0,19.00,18.80,"2,003,229",Upward
232,STA,06-06,06-07,2,18.70,18.70,0.00%,0,0,19.00,18.50,"6,930,076",Downward
233,STA,06-08,06-08,1,19.00,19.00,0.00%,0,0,19.10,18.60,"5,394,096",Upward
234,STA,06-09,06-12,2,18.80,18.50,-1.60%,-3,-2,19.00,18.40,"10,408,367",Downward


In [31]:
df_tmp = price.query('name == @name_down')
df_tmp.describe().round(2)

,price,qty,maxp,minp
count,21.00,21.00,21.00,21.00
mean,18.86,3768326.33,19.10,18.60
std,0.21,1467456.81,0.19,0.22
min,18.50,1715946.00,18.80,18.10
25%,18.70,2752929.00,19.00,18.50
50%,18.80,3229019.00,19.00,18.60
75%,19.00,4935652.00,19.20,18.80
max,19.30,6686234.00,19.60,19.00


In [32]:
mask_p = df['to_md'] == to_mmdd
summary = df[mask_p][cols].sort_values(['pct','trend'],ascending=[False,False])

In [33]:
file_name = 'summary-240.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
one_file = one_path + file_name
osd_file = osd_path + file_name

summary[cols].to_csv(output_file, header=True, index=False)
summary[cols].to_csv(data_file, header=True, index=False)
summary[cols].to_csv(one_file, header=True, index=False)
summary[cols].to_csv(osd_file, header=True, index=False)

In [34]:
sql = """
SELECT name, date, price, qty, maxp, minp
FROM price 
WHERE date >= '%s' 
ORDER BY name, date"""
sql = sql % (today)
print(sql)


SELECT name, date, price, qty, maxp, minp
FROM price 
WHERE date >= '2023-06-12' 
ORDER BY name, date


In [35]:
df = pd.read_sql(sql, const)
df

,name,date,price,qty,maxp,minp
0,ACE,2023-06-12,1.99,17077218,2.02,1.98
1,ADVANC,2023-06-12,218.00,3391263,219.00,216.00
2,AEONTS,2023-06-12,202.00,62902,202.00,200.00
3,AH,2023-06-12,35.25,7101473,36.25,34.75
4,AIE,2023-06-12,1.50,398959,1.52,1.50
...,...,...,...,...,...,...
187,WHAIR,2023-06-12,7.25,135404,7.30,7.20
188,WHART,2023-06-12,11.20,1052392,11.20,11.00
189,WHAUP,2023-06-12,3.86,473970,3.86,3.82
190,WICE,2023-06-12,8.85,1495003,8.95,8.75
